In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2
import os
import keras
from numpy.random import randint
from keras.optimizers import adam_v2
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import LeakyReLU
import tensorflow as tf
import numpy as np
import sys
from numpy import argmax
from keras.preprocessing.image import ImageDataGenerator
import random
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
path_normal = '/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/train_test/train_normal/'
path_cap = '/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/train_test/train_cap/'
path_covid = '/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/train_test/train_covid/'

In [ ]:
normal_image = []
cap_image = []
covid_image = []
normal_files = os.listdir(path_normal)
cap_files = os.listdir(path_cap)
covid_files = os.listdir(path_covid)

In [ ]:
print('/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/normal/' + normal_files[0])
print(len(normal_files))
print(len(cap_files))
print(len(covid_files))

/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/normal/normal034_97.jpg
8003
1323
6633


In [ ]:
shuffled_indices_cap = np.random.permutation(len(cap_files))
shuffled_indices_covid = np.random.permutation(len(covid_files))
shuffled_indices_normal = np.random.permutation(len(normal_files))
indices_cap = shuffled_indices_cap[: 900]
indices_covid = shuffled_indices_covid[: 900]
indices_normal = shuffled_indices_normal[: 900]
print(indices_cap[0], len(indices_cap))
print(indices_covid[0], len(indices_covid))
print(indices_normal[0], len(indices_normal))

1055 900
707 900
331 900


In [ ]:
count = 0
for idx in indices_normal:
  file = normal_files[idx]
  count = count + 1
  if count % 200 == 0:
    print(count)
  temp = path_normal + file
  img = cv2.imread(temp)
  img = np.array(img)
  img = (img/255).astype("float32")
  normal_image.append(img)
del indices_normal
del normal_files
print(len(normal_image))

count = 0
for idx in indices_cap:
  file = cap_files[idx]
  count = count + 1
  if count % 200 == 0:
    print(count)
  temp = path_cap + file
  img = cv2.imread(temp)
  img = np.array(img)
  img = (img/255).astype("float32")
  cap_image.append(img)
del indices_cap
del cap_files
print(len(cap_image))

count = 0
for idx in indices_covid:
  count = count + 1
  file = covid_files[idx]
  if count % 200 == 0:
    print(count)
  temp = path_covid + file
  img = cv2.imread(temp)
  img = np.array(img)
  img = (img/255).astype("float32")
  covid_image.append(img)
del indices_covid
del covid_files
print(len(covid_image))

200
400
600
800
900
200
400
600
800
900
200
400
600
800
900


In [ ]:
X = []
Y = []
for img in normal_image:
  X.append(img)
  Y.append([1, 0, 0])
del normal_image

for img in cap_image:
  X.append(img)
  Y.append([0, 1, 0])
del cap_image

for img in covid_image:
  X.append(img)
  Y.append([0, 0, 1])
del covid_image

np.random.seed(200)
np.random.shuffle(X) 
np.random.seed(200)
np.random.shuffle(Y)

In [ ]:
datagen1 = ImageDataGenerator(featurewise_center=True)
datagen1.fit(X)
mean = datagen1.mean
X = X-mean
print("mean value",mean)
del mean
X = np.reshape(X, (len(X),512,512,3))
print(X.shape)
Y = np.array(Y)

mean value [[[0.0237037 0.0237037 0.0237037]]]
(2700, 512, 512, 3)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
model1 = Sequential([
    EfficientNetB5(include_top=False, weights='imagenet', input_shape=(512,512,3)),
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(1024,activation='tanh'),
    Dense(3, activation='softmax')
])
model1.build(input_shape=(None, 512,512,3))
model1.summary()
model1.compile(loss='categorical_crossentropy', optimizer=adam_v2.Adam(learning_rate=0.0001, beta_1=0.5, decay=5e-2), metrics=['accuracy'])

115277824/115263384 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb5 (Functional)  (None, 16, 16, 2048)      28513527  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3075      
Total params: 30,614,778
Trainable params: 30,442,035
Non-trainable params: 172,743
_________________________________________________________________


In [ ]:
model1.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    batch_size=8,
    epochs=5,
    shuffle=True
)

Epoch 1/5
270/270 [==============================] - 2216s 8s/step - loss: 0.5797 - accuracy: 0.7722 - val_loss: 1.1656 - val_accuracy: 0.3574
Epoch 2/5
270/270 [==============================] - 2127s 8s/step - loss: 0.3708 - accuracy: 0.8620 - val_loss: 1.0333 - val_accuracy: 0.4481
Epoch 3/5
270/270 [==============================] - 2109s 8s/step - loss: 0.3132 - accuracy: 0.8782 - val_loss: 0.9996 - val_accuracy: 0.5093
Epoch 4/5
270/270 [==============================] - 2133s 8s/step - loss: 0.2853 - accuracy: 0.8944 - val_loss: 0.4622 - val_accuracy: 0.7981
Epoch 5/5
270/270 [==============================] - 2131s 8s/step - loss: 0.2526 - accuracy: 0.9116 - val_loss: 0.2744 - val_accuracy: 0.8963


In [ ]:
model1.save_weights("/content/drive/MyDrive/IndividualProject/SPGC-COVID/Results/0.2/three_class_EfficientNet_0.2/three_class_EfficientNet_0.2.ckpt")